# Scraping with Selenium

A lot of modern websites relies on Javascript to navigate dynamically in the content. However the usual Python web scrapers (like `requests`) are not able to execute javascript. Since then they are struggling in getting the content of dynamic web pages.

Selenium is THE solution for tackling this problem. Initially it has been created to automate tests on websites. It will open your browser _for real_ and allow you to simulate human interactions in website through Python commands.

For example it can be useful when information is accessible by clicking on buttons (which is not possible with `requests` and `beautifulsoup`).

### Install Selenium according to this manual

https://selenium-python.readthedocs.io/installation.html#downloading-python-bindings-for-selenium/bin

*NB: On Linux, put your `geckodriver` (the downloaded extension) in the equivalent path on your machine into `/home/<YOUR_NAME>/.local/bin/`*

In [1]:
# The selenium.webdriver module provides all the implementations of WebDriver
# Currently supported are Firefox, Chrome, IE and Remote. The `Keys` class provides keys on
# the keyboard such as RETURN, F1, ALT etc.
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By

### Search engine simulation

We will simulate a query on the official Python website by using the search bar.

In [2]:
# Here, we create instance of Firefox WebDriver.
driver = webdriver.Firefox()

# The driver.get method will lead to a page given by the URL. WebDriver will wait until the page is fully
# loaded (i.e. the "onload" event has been triggered) before returning the control to your script.
# It should be noted that if your page uses a lot of AJAX calls when loading, WebDriver may not know
# when it was fully loaded.
driver.get("http://www.python.org")

# The following line is a statement confirming that the title contains the word "Python".
assert "Python" in driver.title

# WebDriver offers a method `find_element` that aims to search for item based on attributes
# For example, the input text element can be located by its name attribute by
# using the attribute `name` with the value `q`
elem = driver.find_element(By.NAME, "q")

# Then we send keys. This is similar to entering keys using your keyboard.
# Special keys can be sent using the `Keys` class imported in line 7 (from selenium.webdriver.common.keys import Keys).
# For security reasons, we will delete any pre-filled text in the input field
# (for example, "Search") so that it does not affect our search results:
elem.clear()
elem.send_keys("pycon")
elem.send_keys(Keys.RETURN)

# After submitting the page, you should get the result if there is one. To ensure that certain results
# are found, make an assertion that ensures that the source page does not contain the word "No results found".
assert "No results found." not in driver.page_source
driver.close()

### Getting the title of all the articles from the homepage of _The New York Times_

First let's open the homepage of the newspaper's website.


In [3]:
url = "https://www.nytimes.com/"

driver = webdriver.Firefox()
driver.get(url)

As you can see, you are facing the famous GDPR banner. Let's accept it in order to access the page!

In [4]:
cookie_button = driver.find_element(By.XPATH, "//button[@data-testid='GDPR-accept']")
cookie_button.click()

NoSuchElementException: Message: Unable to locate element: //button[@data-testid='GDPR-accept']; For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
RemoteError@chrome://remote/content/shared/RemoteError.sys.mjs:8:8
WebDriverError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:188:5
NoSuchElementError@chrome://remote/content/shared/webdriver/Errors.sys.mjs:506:5
dom.find/</<@chrome://remote/content/shared/DOM.sys.mjs:132:16


Now let's get all the titles of the articles by using XPATH and let's store them in a list


In [ ]:
article_titles = driver.find_elements(By.XPATH, "//section[@class='story-wrapper']//h3")
all_titles = []
for title in article_titles:
    all_titles.append(title.text)

all_titles

Here we are ! Let's close the browser then !

In [ ]:
driver.close()

### Exercise

1. Use Selenium for opening the homepage of your favourite newspaper (not the New York Times, too easy)
2. Close the cookie banner (if it appears)
3. Get the link of the first article of the page and open it
4. Print the title and the content of the article

**tip:** [Newspaper3k](https://pypi.org/project/newspaper3k/) is a powerful library for scraping articles from newspapers. Have a look to the `fulltext` method.

In [5]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import newspaper

# Define your favorite newspaper's URL
newspaper_url = "https://www.example.com"  # Replace with the newspaper's URL

# Set up Selenium WebDriver (assuming Chrome WebDriver)
driver = webdriver.Chrome()
driver.get(newspaper_url)

try:
    # Close the cookie banner (if it appears)
    cookie_banner = driver.find_element(By.XPATH, "//div[contains(@class, 'cookie-banner')]")
    if cookie_banner.is_displayed():
        close_button = cookie_banner.find_element(By.XPATH, "//button[contains(@class, 'close-button')]")
        close_button.click()

    # Get the link of the first article
    first_article_link = driver.find_element(By.XPATH, "//a[contains(@class, 'article-link')]")
    article_url = first_article_link.get_attribute("href")

    # Open the first article
    driver.get(article_url)

    # Use newspaper3k to extract the article's title and content
    article = newspaper.Article(article_url)
    article.download()
    article.parse()

    # Print the title and content of the article
    print("Title:", article.title)
    print("Content:\n", article.text)

except Exception as e:
    print("An error occurred:", str(e))

finally:
    driver.quit()


An error occurred: Message: no such element: Unable to locate element: {"method":"xpath","selector":"//div[contains(@class, 'cookie-banner')]"}
  (Session info: chrome=117.0.5938.88); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
	GetHandleVerifier [0x00007FF778AB78A2+54818]
	(No symbol) [0x00007FF778A26AD2]
	(No symbol) [0x00007FF7788DDA3B]
	(No symbol) [0x00007FF77891E4FC]
	(No symbol) [0x00007FF77891E67C]
	(No symbol) [0x00007FF778959627]
	(No symbol) [0x00007FF77893EAEF]
	(No symbol) [0x00007FF7789575A2]
	(No symbol) [0x00007FF77893E883]
	(No symbol) [0x00007FF778913691]
	(No symbol) [0x00007FF7789148D4]
	GetHandleVerifier [0x00007FF778E1B9A2+3610402]
	GetHandleVerifier [0x00007FF778E71870+3962352]
	GetHandleVerifier [0x00007FF778E69D5F+3930847]
	GetHandleVerifier [0x00007FF778B53656+693206]
	(No symbol) [0x00007FF778A31638]
	(No symbol) [0x00007FF778A2D944]
	(No symbol) [